<h1> Functionality 3 - Shortest Ordered Route</h1>

In [326]:
#import libraries
import pandas as pd
import csv
import networkx as nx
import matplotlib.pyplot as plt
from networkx.algorithms.shortest_paths.weighted import single_source_dijkstra
import collections
import heapq
from collections import defaultdict
from itertools import groupby

<h2> Read the data </h2>

In [106]:
#check the content of the files by printing only few lines
line_num = 0
with open("USA-road-d.CAL.gr", encoding='utf-8') as file:
    for i in file:
        if (line_num < 8):
            print(i)
            line_num += 1

c 9th DIMACS Implementation Challenge: Shortest Paths

c http://www.dis.uniroma1.it/~challenge9

c TIGER/Line graph USA-road-d.CAL

c

p sp 1890815 4657742

c graph contains 1890815 nodes and 4657742 arcs

c

a 1 1048577 456



In [107]:
#consider only the lines after 7th line, split the values and store the necessary data in lists
node1 = []
node2 = []
distance = []
line_num = 0
with open("USA-road-d.CAL.gr", encoding='utf-8') as file:
    for i in file:
        if (line_num > 6):
            parts = i.split()
            node1.append(parts[1])
            node2.append(parts[2])
            distance.append(parts[3])
        line_num += 1

In [108]:
#create a dataframe of the lists with another column (ie. network distance = 1) 
df = pd.DataFrame(
    {'source': node1,
     'target': node2,
     'distance': distance,
     'weight' : 1
    })

In [109]:
#check how the dataframe looks
df

,source,target,distance,weight
0,1,1048577,456,1
1,1048577,1,456,1
2,2,1048578,2389,1
3,1048578,2,2389,1
4,3,1048579,358,1
...,...,...,...,...
4657737,1890815,1048576,5642,1
4657738,989247,1048576,3212,1
4657739,1048576,989247,3212,1
4657740,1048572,1890810,7230,1


In [110]:
#drop the column with distance as we will use network distance
df = df.drop(columns = ["distance"])

In [111]:
#use networkx to create an edgelist (for validating the results)
G = nx.from_pandas_edgelist(df,'source','target', edge_attr='weight')

<h2> The shortest route function </h2>

In [116]:
#create a class called Graph where 
#self.edges is a dict of all possible next nodes e.g. {'1': ['4', '3', '8', '9'], ...}
#self.weights has all the weights between two nodes, with the two nodes as a tuple as the key e.g. {('1', '2'): 1, ('5', '9'): 1, ...}

class Graph():
    def __init__(self):
        self.edges = defaultdict(list)
        self.weights = {}
    
    def add_edge(self, from_node, to_node, weight):
        # assuming edges are bi-directional
        self.edges[from_node].append(to_node)
        self.edges[to_node].append(from_node)
        self.weights[(from_node, to_node)] = weight
        self.weights[(to_node, from_node)] = weight

In [117]:
#call the Graph class
graph = Graph()

In [123]:
#create an edge list with source, target and network distance
edge_list = []
for i,j in df.iterrows():
    edge_list.append((j[0],j[1],j[2]))

In [124]:
#add the edge list to the graph
for edge in edge_list:
    graph.add_edge(*edge)

In [260]:
#shortest route function
def dijsktra(graph, initial, end):
    # shortest paths is a dict of nodes
    # whose value is a tuple of (previous node, weight)
    shortest_paths = {initial: (None, 0)}
    current_node = initial
    visited = set()
    
    while current_node != end:
        visited.add(current_node)
        destinations = graph.edges[current_node]
        weight_to_current_node = shortest_paths[current_node][1]

        for next_node in destinations:
            weight = graph.weights[(current_node, next_node)] + weight_to_current_node
            if next_node not in shortest_paths:
                shortest_paths[next_node] = (current_node, weight)
            else:
                current_shortest_weight = shortest_paths[next_node][1]
                if current_shortest_weight > weight:
                    shortest_paths[next_node] = (current_node, weight)
    
        next_destinations = {node: shortest_paths[node] for node in shortest_paths if node not in visited}
        if not next_destinations:
            return "Route Not Possible"
        # next node is the destination with the lowest weight
        current_node = min(next_destinations, key=lambda k: next_destinations[k][1])
        

    # Work back through destinations in shortest path
    path = []
    weights = []
    while current_node is not None:
        path.append(current_node)
        next_node = shortest_paths[current_node][0]
        weights.append(shortest_paths[current_node][1])
        current_node = next_node
    # Reverse path
    path = path[::-1]
    return weights[0],path

In [370]:
#take the source input
source = input()

21


In [371]:
#take the destination input
list_of_nodes_to_dest = []
list_of_nodes_to_dest.append(source)
while True:
    inp = input()
    if inp == "":
        break
    else:
        list_of_nodes_to_dest.append(inp)

22
23
24



In [380]:
#pack the graph, source input and destination input into a tuple ie. (graph,source,destination)
res = [(graph ,list_of_nodes_to_dest[i], list_of_nodes_to_dest[i + 1])
        for i in range(len(list_of_nodes_to_dest) - 1)] 

In [381]:
#call the shortest route function to find the shortest path and weight between each set of nodes
arr = []
for tripple in res:
    for i in dijsktra(*tripple):
        arr.append(i)
    print(tripple[1] +" -> " +tripple[2], dijsktra(*tripple))

21 -> 22 (12, ['21', '20', '19', '18', '1048591', '155', '139', '1048683', '124', '16', '17', '1048592', '22'])
22 -> 23 (11, ['22', '1048593', '2319', '1048641', '80', '66', '1048629', '65', '1048628', '1048625', '61', '23'])
23 -> 24 (1, ['23', '24'])


In [374]:
#run the below function to find the shortest path for a set of nodes(for validating the results)
single_source_dijkstra(G,'21','22')

(12,
 ['21',
  '20',
  '19',
  '18',
  '1048591',
  '155',
  '139',
  '1048683',
  '124',
  '16',
  '17',
  '1048592',
  '22'])

In [383]:
print(arr)

[12, ['21', '20', '19', '18', '1048591', '155', '139', '1048683', '124', '16', '17', '1048592', '22'], 11, ['22', '1048593', '2319', '1048641', '80', '66', '1048629', '65', '1048628', '1048625', '61', '23'], 1, ['23', '24']]


In [384]:
#store the resultant path and weight separately
res_path = [arr[i] for i in range(len(arr)) if i % 2 != 0] 
res_weight = [arr[i] for i in range(len(arr)) if i % 2 == 0] 

In [388]:
#flatten the list of lists of path
flattened_list = []
for x in res_path:
    for y in x:
        flattened_list.append(y)

In [390]:
#clean up the path and add up the weights of the set of nodes
res_path = [x[0] for x in groupby(flattened_list)]
res_weight  = sum(res_weight)

In [391]:
print("The shortest route from " + source +" to " + list_of_nodes_to_dest[-1] +" is:\n" +str(res_path))
print("The total weight is " +str(res_weight))

The shortest route from 21 to 24 is:
['21', '20', '19', '18', '1048591', '155', '139', '1048683', '124', '16', '17', '1048592', '22', '1048593', '2319', '1048641', '80', '66', '1048629', '65', '1048628', '1048625', '61', '23', '24']
The total weight is 24
